# MBAM model reduction for the MWC model of BK
Model taken from Aldrich 2002 (Equation 1) <br>
Initial parameters taken from Miranda 2013 PNAS, Suppl fig 5 <br>
Algorithm as in Transtrum 2015 (Equation 49-51) - except for signs

The code below iteratively finds limits (boundaries) and stiff nonlinear combinations of sloppy parameters for the MWC model of BK, while preserving the model output function $P_{open}(V, [Ca] \ | \ \theta)$.

The iterative model reduction proceeds as follows:
+ $\theta = \{L_0, z_L, J_0, z_J, K_d, C, D, E \}$
    * $z_L -> 0$
    
    
+ $\phi^1 = \{L_0, J_0, z_J, K_d, C, D, E \}$
    * $L_0^{1/4} \rightarrow 0, \quad C \rightarrow \infty, \quad D \rightarrow \infty, \quad E \rightarrow 0 $
    

+ $\phi^2 = \{J_0, z_J, K_d, L_0^{1/4}C, L_0^{1/4}D, L_0^{-1/4}E \}$ 
    * $L_0^{1/4}C \rightarrow 0, \quad L_0^{-1/4}E \rightarrow \infty $
    
    
+ $\phi^3 = \{J_0, z_J, K_d, CE, L_0^{1/4}D \}$ 
    * $J_0 \rightarrow 0, \quad L_0^{1/4}D \rightarrow \infty $
 
 
+ $\phi^4 = \{J_0L_0^{1/4}D, z_J, K_d, CE  \}$ 
    * $K_d \rightarrow \infty, \quad CE \rightarrow \infty $
 
 
+ $\phi^5 = \{J_0L_0^{1/4}D, z_J, CEK_d^{-1}  \}$ 

The $P_{open}(V, [Ca] \ | \ \phi^5)$ function is an excellent approximation of the original (way below experimental error), despite having only 3 parameters. Each of the parameters are meaningful, the first one describing the slope of the voltage sensitivity, the second the middle point of the voltage sensitivity and the third the calcium sensitivity.
Note that the 4 parameter-model ($\phi^4$) describes both the voltage and the calcium sensitivity curves as two-parameter sigmoids. The allosteric MWC model introduces nonlinear interaction between these two sensitivity curves.

In [73]:
using NBInclude #Includes for ipynb-s
using JLD # Julia data I/O
nbinclude("MBAM.ipynb")
nbinclude("BK_functions/BK_functions.ipynb")


BK_cost (generic function with 1 method)

In [75]:
# Create grid to evaluate over
using Iterators
Ca=([0, 0.7, 4, 12, 22, 55, 70, 95]*1e-6)
V=collect(-100:25:200)*1e-3

x_grid = Array(Any, (length(Ca)*length(V),))
iterind = 0;
for j1 in product(Ca, V)
    iterind += 1;
    x_grid[iterind] = collect(j1)
end
size(x_grid[1])

x= 
size([0.0,-0.1])

(2,)

In [76]:
# Single MBAM call that runs through all iterations of BK model reduction with automated model reduction
phi0 = [2.2e-6, 0.42, 0.1026, 0.58, 39*1e-6, 6.16,30.4,2.0];
outp = MBAM(BK_simulator, phi0, x_grid, model_iters=[10], move_dir=[-1], boundary_time=10, verbose=1);


x_loc[1] = [0.0,-0.1]
Model iteration 10, initial values:
    D =  -8.30e-20  -2.45e-21   6.59e-27   5.31e-21   8.99e-21   9.21e-21   2.68e-06   8.23e-04
theta =     -13.03      -0.87      -2.28      -0.54     -10.15       1.82       3.41       0.69
    v =   2.43e-01   1.11e-01   5.08e-01   1.61e-01  -2.00e-04  -9.11e-02  -5.26e-01   5.99e-01


Model iteration 10, final values:
    D =  -8.30e-20  -2.45e-21   6.59e-27   5.31e-21   8.99e-21   9.21e-21   2.68e-06   8.23e-04
theta =     -19.74     -10.16      -2.96      -0.55     -10.29       2.19       5.95       0.11
    v =  -9.41e-04  -1.00e+00  -6.92e-05   4.06e-06  -7.45e-06   6.93e-05   3.15e-04  -8.16e-05


size(x_loc[1]) = (2,)
data_dim = (2,)






(model_reduced[n1])[3] = Dict{Any,Any}(phi7=>phi6,phi3=>phi2,phi6=>phi5,phi1=>phi1,phi5=>phi4,phi4=>phi3,phi8=>phi7)
phi_red = [2.6731721461807124e-9,0.05181891717272583,0.5769498103804866,3.397111341189323e-5,8.935213114698747,383.7533390611119,1.1162780704588713]


In [80]:
log(outp[5])

7-element Array{Float64,1}:
 -19.74
  -2.96
  -0.55
 -10.29
   2.19
   5.95
   0.11

In [ ]:
outp[8][1][2] # First reduced model

In [ ]:
nbinclude("MBAM.ipynb")
phi_boundary = exp(BK_reduce([-19.74, -10.16,      -2.96,      -0.55,     -10.29,       2.19,       5.95,       0.11], model_id=11))
outp = MBAM(BK_cost, phi_boundary, BK_simulator, model_iters=11, move_dir=-1, boundary_time=10, verbose=1);

Model iteration 11, initial values:
    D =   6.30e-06   1.53e-04   3.21e-02   6.72e-02   3.55e-01   8.12e+00   3.62e+01
theta =     -19.74      -2.96      -0.55     -10.29       2.19       5.95       0.11
    v =   9.42e-01  -6.99e-03   8.88e-03   1.08e-02  -1.49e-01  -2.46e-01   1.73e-01


Model iteration 11, final values:
    D =   6.30e-06   1.53e-04   3.21e-02   6.72e-02   3.55e-01   8.12e+00   3.62e+01
theta =     -35.33      -2.93      -0.60     -10.35       5.39       9.91      -3.22
    v =  -9.21e-01   9.86e-05  -4.34e-04  -4.31e-04   2.22e-01   2.31e-01  -2.23e-01


phi_comb = Any[phi1^0.25*phi5^1.0*phi6^1.0*phi7^1.0,phi1^(-0.25)*phi5^1.0*phi6^1.0*phi7^1.0,phi1^0.25*phi5^(-1.0)*phi6^1.0*phi7^1.0,phi1^(-0.25)*phi5^(-1.0)*phi6^1.0*phi7^1.0,phi1^0.25*phi5^1.0*phi6^(-1.0)*phi7^1.0,phi1^(-0.25)*phi5^1.0*phi6^(-1.0)*phi7^1.0,phi1^0.25*phi5^(-1.0)*phi6^(-1.0)*phi7^1.0,phi1^(-0.25)*phi5^(-1.0)*phi6^(-1.0)*phi7^1.0,phi1^0.25*phi5^1.0*phi6^1.0*phi7^(-1.0),phi1^(-0.25)*phi5^1.0*phi6^1.

In [18]:
save("BK_results/geodesic_$(timestamp()).jld", "outp", )

LoadError: LoadError: cannot write a pointer to JLD file
while loading In[18], in expression starting on line 1

In [26]:
outp[8][1][1]

                                                                              
                                                 /                            
                                                 |           38.1727608645031*
                                            phi1*|phi2*phi6*e                 
                                                 \                            
------------------------------------------------------------------------------
                                                                              
     /                                                              38.1727608
     |           38.1727608645031*phi3*x2   phi2*phi5*phi6*phi7*x1*e          
phi1*|phi2*phi6*e                         + ----------------------------------
     \                                                            phi4        

                                                                         4    
                                  38.1727608645031*

In [11]:
vn = v/norm(v)
norm(vn[abs(vn).<5e-2])<1e-3

false

In [ ]:
# Single MBAM call that runs through all iterations of BK model reduction
x_0 = [2.2e-6, 0.42, 0.1026, 0.58, 39*1e-6, 6.16,30.4,2.0]
@time (x, cost, v, D, x_red1) = MBAM(BK_cost, x_0, BK_reduce, model_iters=collect(10:14), move_dir=[-1,-1,1,1,1], boundary_time=10, verbose=1);


### Iterative MBAM calls
(Old)

In [ ]:
#Run MBAM on BK model (inital iteration)
x_0 = [2.2e-6, 0.42, 0.1026, 0.58, 39*1e-6, 6.16,30.4,2.0]
@time (x, cost, v, D, x_red1) = MBAM(BK_cost, x_0, BK_reduce, model_iters=10, boundary_time=10, verbose=0);
@save "BK_results/geodesic_iter_1$(timestamp()).jld"

In [ ]:
#Run MBAM on BK model (second iteration)
x_red1 = BK_reduce(x, model_id=11)
(x, cost, v, D, x_red2) = MBAM(BK_cost, exp(x_red1), BK_reduce, model_iters=11, boundary_time=10, verbose=0);
@save "BK_results/geodesic_iter_2$(timestamp()).jld"

In [ ]:
#Run MBAM on BK model (third iteration)
x_red2 = BK_reduce(x, model_id=12)
@time (x, cost, v, D, x_red3) = MBAM(BK_cost, exp(x_red2), BK_reduce, model_iters=12, boundary_time=10, verbose=1); 
@save "BK_results/geodesic_iter_3$(timestamp()).jld"

In [ ]:
#Run MBAM on BK model (fourth iteration)
@time (x, cost, v, D, x_red4) = MBAM(BK_cost, exp(x_red3), BK_reduce, model_iters=13, boundary_time=10, verbose=0); 
@save "BK_results/geodesic_iter_4$(timestamp()).jld"
# Doesn't really seem to converge anymore

In [ ]:
#Run MBAM on BK model (fifth iteration)
@time (x, cost, v, D, x_red5) = MBAM(BK_cost, exp(x_red4), BK_reduce, model_iters=14, boundary_time=10, verbose=0); 
@save "BK_results/geodesic_iter_5$(timestamp()).jld"

In [ ]:
using Plots
x_0 = [2.2e-6, 0.42, 0.1026, 0.58, 39*1e-6, 6.16,30.4,2.0]

#Original model at x_0
Plots.plot(BK_simulator(x_0,model_id=0)')

In [ ]:
# Reduced model at the boundary
Plots.plot(BK_simulator(exp(x),model_id=14)')